<table class="table table-bordered">
    <tr>
        <th style="text-align:center; width:25%"><img src='https://www.nus.edu.sg/images/default-source/base/logo.png' style="width: 250px; height: 125px; "></th>
        <th style="text-align:center;"><h1>Applied Machine Learning</h1><h2>Mini Project </h2><h3></h3></th>
    </tr>
</table>

In [ ]:
# For Google Colaboratory
import sys, os
if 'google.colab' in sys.modules:
    # mount google drive
    from google.colab import drive
    drive.mount('/content/gdrive')
    path_to_file = '/content/gdrive/My Drive/Applied_ML/Practical' # Please adjust the path accordingly 
    os.chdir(path_to_file)
    !pwd

Mounted at /content/gdrive
/content/gdrive/My Drive/Applied_ML/Practical


In [ ]:
# Import the requried packages
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import linear_model, neighbors, tree, svm, ensemble
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, confusion_matrix, classification_report
from sklearn.model_selection import cross_val_score, cross_validate

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense



You will be using the bank marketing campaign data to build classification model. The dataset (`bank.csv`) gives you information about a marketing campaign done by a financial institution. Detailed information (i.e. column description) is provided below. We are trying to predict whether the customer/client will deposit the money into the bank or not?

 * **age :**  age in years
 * **job :**  type of job
 * **marital :**  marital status 
 * **education :**  education background 
 * **default :**  has credit in default? 
 * **balance :**  Balance of the individual
 * **housing :**  has housing loan? 
 * **loan :**  has personal loan? 
 * **contact :**  contact communication type 
 * **day :**  last contact day of the week 
 * **month :**  last contact month of year 
 * **duration :**  last contact duration, in seconds 
 * **campaign :**  number of contacts performed during this campaign and for this client
 * **pdays :**  number of days that passed by after the client was last contacted from a previous campaign 
 * **previous :**  number of contacts performed before this campaign and for this client
 * **poutcome :**  outcome of the previous marketing campaign 
 * <font color='red'> **deposit :** has the client subscribed a term deposit? This is the TARGET variable </font> 

We have provided data preprocessing codes per below

In [ ]:
# Loading the data
bank = pd.read_csv('bank.csv')
print(bank.head())
# Encode the categorical data into numbers
bank_cat = bank.select_dtypes(['object']).copy()
for col in bank_cat:
    print(col, ':')
    codes, uniques = pd.factorize(bank_cat[col], sort=True)
    bank[col]=codes
    print(uniques)
print(bank.head())
    
# Set the "deposit" as target/model output and the rest features as model inputs
y = bank['deposit']
X = bank.drop(['deposit'], axis=1)

# Split the data into training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

#Sacaling the data into a smaller range (-3 to +3)
mean = X_train.mean()
std = X_train.std()

X_train = (X_train - mean) / std
X_test = (X_test - mean) / std


   age         job  marital  education default  balance housing loan  contact  \
0   59      admin.  married  secondary      no     2343     yes   no  unknown   
1   56      admin.  married  secondary      no       45      no   no  unknown   
2   41  technician  married  secondary      no     1270     yes   no  unknown   
3   55    services  married  secondary      no     2476     yes   no  unknown   
4   54      admin.  married   tertiary      no      184      no   no  unknown   

   day month  duration  campaign  pdays  previous poutcome deposit  
0    5   may      1042         1     -1         0  unknown     yes  
1    5   may      1467         1     -1         0  unknown     yes  
2    5   may      1389         1     -1         0  unknown     yes  
3    5   may       579         1     -1         0  unknown     yes  
4    5   may       673         2     -1         0  unknown     yes  
job :
Index(['admin.', 'blue-collar', 'entrepreneur', 'housemaid', 'management',
       'retired', 

In [ ]:
X_train

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome
10713,0.311590,-1.399276,-0.323496,2.287973,-0.119149,0.150938,-0.942781,-0.389825,-0.598686,1.463949,0.806711,-0.939404,0.531137,-0.479882,-0.375551,0.518328
8431,1.233067,-1.399276,-0.323496,-1.715942,-0.119149,-0.415907,1.060573,-0.389825,1.845607,-0.797574,0.806711,-0.565266,-0.192912,-0.479882,-0.375551,0.518328
9060,0.730443,-0.467874,1.274123,-0.381304,-0.119149,-0.231086,-0.942781,-0.389825,0.623460,1.463949,-0.444320,-0.896235,-0.554937,-0.479882,-0.375551,0.518328
8604,1.400608,-1.399276,-0.323496,-0.381304,-0.119149,-0.025215,1.060573,2.564968,1.845607,-1.273684,0.181196,-0.925014,9.581747,-0.479882,-0.375551,0.518328
2988,0.144049,-0.157407,1.274123,0.953334,-0.119149,3.202795,-0.942781,-0.389825,-0.598686,-1.392711,-1.069836,1.285282,0.893161,1.778920,0.083679,-0.485173
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1099,-1.028739,1.705397,1.274123,-1.715942,-0.119149,-0.308172,-0.942781,-0.389825,-0.598686,1.582977,-0.444320,1.227722,-0.554937,-0.479882,-0.375551,0.518328
2514,-1.531363,1.084462,1.274123,-0.381304,-0.119149,-0.345013,-0.942781,-0.389825,-0.598686,-0.321464,-1.382594,0.349935,-0.192912,-0.479882,-0.375551,0.518328
6637,-0.191033,-0.157407,-0.323496,0.953334,-0.119149,-0.478442,1.060573,-0.389825,-0.598686,-0.559519,0.806711,-0.755213,-0.554937,-0.479882,-0.375551,0.518328
2575,1.568149,-1.399276,-0.323496,-0.381304,-0.119149,-0.445627,-0.942781,-0.389825,-0.598686,0.630757,-1.382594,-0.455902,0.169112,0.375303,0.083679,-0.485173


In [ ]:
y_train

10713    0
8431     0
9060     0
8604     0
2988     1
        ..
1099     1
2514     1
6637     0
2575     1
7336     0
Name: deposit, Length: 8929, dtype: int64